In [1]:
model = "indobertweet"

out_path = f"pred/{model}/"

data = {
    1 : {
        "model": "../tensorboard_checkpoint/indobert/2023-04-27_04-48-48_scenario_1/checkpoints/val_epoch=2-validation_loss=0.0542.ckpt",
        "dataset":"original/1.csv",
    },
    3 : {
        "model": "../tensorboard_checkpoint/indobert/2023-04-27_04-48-48_scenario_3/checkpoints/val_epoch=0-validation_loss=0.0793.ckpt",
        "dataset":"original/3.csv",
    },
    5 : {
        "model": "../tensorboard_checkpoint/indobert/2023-04-27_04-48-48_scenario_5/checkpoints/val_epoch=0-validation_loss=0.0691.ckpt",
        "dataset":"original/5.csv",
    },
}

In [2]:
# Load dataset to df
import pandas as pd
for i in data:
    data[i]['df'] = pd.read_csv(data[i]['dataset'])

In [3]:
# Hyperparameters
"""
https://arxiv.org/pdf/1810.04805.pdf

We use a batch size of 32 and fine-tune for 3
epochs over the data for all GLUE tasks. For each
task, we selected the best fine-tuning learning rate
(among 5e-5, 4e-5, 3e-5, and 2e-5)
"""
hyper_params = {
    'model_name': "indolem/indobertweet-base-uncased",
    'seq_length': 256,
    'out_feature': 2,
    'learning_rate': 2e-5,
    'batch_size': 16
}

import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning.pytorch as pl
from transformers import BertForSequenceClassification, BertTokenizerFast
from torchmetrics.classification import F1Score, Accuracy, Recall, Precision


class BERTClassifier(pl.LightningModule):
    def __init__(
        self,
        huggingface_model_name: "str" = "indolem/indobertweet-base-uncased",
        seq_length: "int" = 256,
        out_feature: "int" = 2,
        pad_sequence: "bool" = True,
    ):
        super().__init__()
        self.seq_length = seq_length
        self.pad_sequence = pad_sequence
        self.tokenizer = BertTokenizerFast.from_pretrained(huggingface_model_name)
        self.huggingface_model = BertForSequenceClassification.from_pretrained(
            huggingface_model_name,
            num_labels=out_feature,
            problem_type="multi_label_classification",
        ).to(self.device)

        self.f1_scorer = F1Score(task="multiclass", num_classes=out_feature)
        self.accuracy_scorer = Accuracy(task="multiclass", num_classes=out_feature)
        self.precision_scorer = Precision(task="multiclass", num_classes=out_feature)
        self.recall_scorer = Recall(task="multiclass", num_classes=out_feature)

    def _forward_huggingface_tokenizers(self, x: "list[str]"):
        for sentence in x:
            sentence = f"{sentence}"
            sentence_seq = sentence.split(" ")
            if len(sentence_seq) > self.seq_length:
                sentence_seq = sentence_seq[: self.seq_length]
            if self.pad_sequence:
                while len(sentence_seq) < self.seq_length:
                    sentence_seq.append("[PAD]")
        tokens = self.tokenizer(
            x,
            max_length=512, # Max BERT tokens
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        input_ids = tokens["input_ids"].to(self.device)  # type: ignore
        attention_mask = tokens["attention_mask"].to(self.device)  # type: ignore
        return input_ids, attention_mask

    def forward(self, x: "list[str]") -> "torch.Tensor":
        # Prepare str
        input_ids, attention_mask = self._forward_huggingface_tokenizers(x)
        logits = self.huggingface_model(input_ids=input_ids, attention_mask=attention_mask).logits  # type: ignore
        return logits

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        pred = self.forward(x[0])
        loss = F.cross_entropy(pred, y)
        self.log("training_loss", loss)
        f1_score = self.f1_scorer(pred, y)
        self.log("training_f1", f1_score, prog_bar=True)
        accuracy = self.accuracy_scorer(pred, y)
        self.log("training_accuracy", accuracy, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        pred = self.forward(x[0])
        loss = F.cross_entropy(pred, y)
        self.log("validation_loss", loss, prog_bar=True)
        f1_score = self.f1_scorer(pred, y)
        self.log("validation_f1", f1_score, prog_bar=True)
        accuracy = self.accuracy_scorer(pred, y)
        self.log("validation_accuracy", accuracy, prog_bar=True)
        precision = self.precision_scorer(pred, y)
        self.log("validation_precision", precision, prog_bar=True)
        recall = self.recall_scorer(pred, y)
        self.log("validation_recall", recall, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        pred = self.forward(x[0])
        loss = F.cross_entropy(pred, y)
        self.log("test_loss", loss)
        f1_score = self.f1_scorer(pred, y)
        self.log("test_f1", f1_score, prog_bar=True)
        accuracy = self.accuracy_scorer(pred, y)
        self.log("test_accuracy", accuracy, prog_bar=True)
        precision = self.precision_scorer(pred, y)
        self.log("test_precision", precision, prog_bar=True)
        recall = self.recall_scorer(pred, y)
        self.log("test_recall", recall, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=hyper_params['learning_rate'])
        return optimizer

model_module = BERTClassifier
def create_model():
    return model_module(hyper_params['model_name'], hyper_params['seq_length'], hyper_params['out_feature'])


In [4]:
import os
from tqdm import tqdm

for i in data:
    print(f"Processing scenario {i}")

    if not os.path.exists(out_path):
        os.mkdir(out_path)

    state_dict = torch.load(data[i]['model'], map_location="cpu")['state_dict']
    model = create_model()
    model.load_state_dict(state_dict)
    model.eval()
    text_data = data[i]['df']['tweet'].to_list()
    predictions = []
    
    with torch.no_grad():
        for text in tqdm(text_data):
            logits = model([f"{text}"])
            predictions.append(logits)

    predictions = torch.cat(predictions, 0)
    softmax_tensor = F.softmax(predictions)
    argmax_tensor = torch.argmax(softmax_tensor, 1)
    np_pred = argmax_tensor.cpu().numpy()

    df_pred = data[i]['df'].copy()
    df_pred['prediction'] = np_pred
    df_pred.to_csv(f"{out_path}/{i}.csv", index=False)

Processing scenario 1


Some weights of the model checkpoint at indolem/indobertweet-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/ind

Processing scenario 3


Some weights of the model checkpoint at indolem/indobertweet-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/ind

Processing scenario 5


Some weights of the model checkpoint at indolem/indobertweet-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/ind